### 활성화 함수
- __init__() 함수에서는 모델에서 사용될 모듈(nn.Linear 등)과 activation function(활성화 함수) 등을 정의함
- forward() 함수에서 실행되어야 하는 연산에 활성화 함수도 적용하면 됨
- 주요 활성화 함수
    - 시그모이드 함수 : nn.Sigmoid()
    - ReLU 함수 : nn.ReLU()
    - Leaky ReLU 함수 : nn.LeakyReLU()

In [1]:
import torch
import torch.nn as nn

class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.linear = nn.Linear(in_features=input_dim, out_features=output_dim)
        self.activation = nn.Sigmoid() # 시그모이드 함수
    def forward(self, x):
        return self.activation(self.linear(x))

In [2]:
x = torch.ones(4)
y = torch.zeros(3)
model = LinearRegressionModel(4, 3)
loss_function = nn.MSELoss()

In [3]:
learning_rate = 0.01
nb_epochs = 1000
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(nb_epochs + 1):
    
    y_pred = model(x)
    loss = loss_function(y_pred, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [4]:
print(loss)
for param in model.parameters():
    print(param)

tensor(0.0243, grad_fn=<MseLossBackward0>)
Parameter containing:
tensor([[-0.7735, -0.2206, -0.0008,  0.1168],
        [-0.6442, -0.4289, -0.5716,  0.1211],
        [-0.3785, -0.3623, -0.7004,  0.0541]], requires_grad=True)
Parameter containing:
tensor([-0.7881, -0.3031, -0.2112], requires_grad=True)


### 다층 레이어 구현
> raw level 로 구현해본 후, 좀더 유용한 클래스를 알아보기로

- input layer -> hidden layer -> output layer 순으로 순차적으로 작성해주면 됨
    - 내부 행렬곱 조건만 유의해주면 됨

- activation function 적용은 output layer 에는 적용하지 않는 것이 일반적임

In [5]:
import torch
import torch.nn as nn

class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, 10)
        self.linear2 = nn.Linear(10, 10)
        self.linear3 = nn.Linear(10, 10)
        self.linear4 = nn.Linear(10, output_dim)
        self.activation = nn.LeakyReLU(0.1)
        
    def forward(self, x):
        # |x| = (input_dim, output_dim)
        hidden = self.activation(self.linear1(x)) # |hidden| = (input_dim, 5)
        hidden = self.activation(self.linear2(hidden)) # |hidden| = (5, 5)
        hidden = self.activation(self.linear3(hidden)) # |hidden| = (5, 5)
        y = self.linear4(hidden) # 마지막 출력에는 activation 함수를 사용하지 않는 것이 일반적임
        return y
        

In [6]:
x = torch.ones(4)
y = torch.zeros(3)
model = LinearRegressionModel(4, 3)
loss_function = nn.MSELoss()

In [7]:
learning_rate = 0.01
nb_epochs = 1000
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(nb_epochs + 1):
    
    y_pred = model(x)
    loss = loss_function(y_pred, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [8]:
print(loss)
for param in model.parameters():
    print(param)

tensor(5.3889e-11, grad_fn=<MseLossBackward0>)
Parameter containing:
tensor([[ 0.0727,  0.0699, -0.4880, -0.4512],
        [ 0.3895,  0.2990,  0.2938,  0.0815],
        [ 0.2743, -0.0419, -0.1630, -0.4318],
        [-0.3889,  0.2126, -0.2284, -0.2245],
        [ 0.0524,  0.2113,  0.2521,  0.3501],
        [ 0.0673,  0.2327, -0.2678,  0.4528],
        [-0.2693,  0.3649,  0.4550,  0.3656],
        [ 0.1441,  0.1624, -0.4174,  0.2337],
        [ 0.2957,  0.3314,  0.4524,  0.4067],
        [ 0.0877, -0.3260,  0.1033,  0.4192]], requires_grad=True)
Parameter containing:
tensor([ 0.2755,  0.3230,  0.4459,  0.2198,  0.3354,  0.4236, -0.4003,  0.2015,
         0.0520,  0.1049], requires_grad=True)
Parameter containing:
tensor([[-0.2284, -0.2767, -0.0242,  0.1591,  0.1088, -0.0907, -0.0245,  0.0584,
         -0.0551,  0.0116],
        [-0.0637, -0.1881,  0.2396,  0.2462,  0.0616,  0.1611,  0.0751,  0.2162,
         -0.2784,  0.2664],
        [ 0.0400,  0.2052, -0.1707, -0.1115,  0.2480, -0.1998

### nn.Sequential

- nn.Sequential 은 순서를 갖는 모듈의 컨테이너를 의미함
- 순차적으로 연산되는 레이어만 있을 경우에는, nn.Sequential을 통해 순서대로 각 레이어를 작성하면 그대로 실행됨
    - 중간에 activation function이 적용된다면, activation function도 순서에 맞게 넣어주면 자동 계산됨

In [9]:
print(x.size(0))
print(y.size(0))

4
3


In [10]:
input_dim = x.size(0)
output_dim = y.size(0)

model = nn.Sequential(
    nn.Linear(input_dim, 10),
    nn.LeakyReLU(0.1),
    nn.Linear(10, 10),
    nn.LeakyReLU(0.1),
    nn.Linear(10, 10),
    nn.LeakyReLU(0.1),
    nn.Linear(10, output_dim)
)

In [11]:
learning_rate = 0.01
nb_epochs = 1000
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(nb_epochs + 1):
    
    y_pred = model(x)
    loss = loss_function(y_pred, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
print(loss)
for param in model.parameters():
    print(param)

tensor(1.4144e-11, grad_fn=<MseLossBackward0>)
Parameter containing:
tensor([[-0.0442,  0.2914,  0.3071,  0.3162],
        [-0.1451,  0.3318, -0.1291, -0.3782],
        [-0.0738, -0.2778, -0.3890, -0.4934],
        [-0.0363, -0.1630, -0.1278,  0.0555],
        [-0.4286, -0.1900,  0.0995, -0.2950],
        [ 0.4898, -0.0046,  0.2333,  0.4315],
        [-0.3636, -0.2509,  0.1210, -0.2384],
        [-0.0533,  0.3700, -0.3006, -0.2195],
        [-0.1318,  0.0449, -0.0167,  0.2088],
        [-0.1904, -0.0896, -0.0158, -0.0880]], requires_grad=True)
Parameter containing:
tensor([-0.4280,  0.0463,  0.3380, -0.0250,  0.1573,  0.3354,  0.4536, -0.3715,
         0.0501,  0.2163], requires_grad=True)
Parameter containing:
tensor([[ 1.1600e-01,  8.2027e-02,  2.1091e-02, -4.7452e-02,  5.3687e-02,
          1.5900e-01,  1.9247e-01, -4.3328e-02, -3.1011e-01, -2.7504e-02],
        [-8.9423e-02,  3.6057e-02, -3.9426e-02,  2.4418e-01,  1.9936e-01,
         -2.4272e-02,  1.3459e-01,  9.1155e-02,  7.9613e

### SGD 방식 구현

- 랜덤하게 데이터를 섞기 위한 함수
    - torch.randperm(n): 0 ~ n -1 까지의 정수를 랜덤하게 섞어서, 순열(배열)을 만들어줌
    - torch.index_select(텐서객체, 차원번호, 인덱스텐서)
        - 차원번호는 예를 들어 , $|x|$ = (3, 4) 에서 0차원에 해당하는 값은 3 (행), 1차원에 해당하는 값은 4(열)
    - 특정 차원의 나열된 인덱스 번호 순서대로, 데이터를 섞어줌

In [21]:
data1 = torch.randn(3, 4)
print(data1)
indices = torch.tensor([1, 2])
print(indices)
print(torch.index_select(data1, 0, indices)) # 행을 기준으로
print(torch.index_select(data1, 1, indices)) # 열을 기준으로

tensor([[ 0.4876,  0.8673,  0.7370, -1.2567],
        [ 0.2807,  0.5982, -1.6942,  0.1102],
        [ 3.0521, -1.9946, -1.5569,  1.0603]])
tensor([1, 2])
tensor([[ 0.2807,  0.5982, -1.6942,  0.1102],
        [ 3.0521, -1.9946, -1.5569,  1.0603]])
tensor([[ 0.8673,  0.7370],
        [ 0.5982, -1.6942],
        [-1.9946, -1.5569]])


In [22]:
x = torch.ones(5000, 10) # 10개의 feature 가 있는 5000개의 데이터셋
y = torch.zeros(5000, 1) # 
learning_rate = 0.01
nb_epochs = 1000
minibatch_size = 256 

In [23]:
input_dim = x.size(-1) # 입력 차원 10
output_dim = y.size(-1) # 출력 차원 1

# 보통 hidden layer는 출력에 가까울 수록 작아지게 설계하는 것이 일반적임(더 좋은 성능)
model = nn.Sequential(
    nn.Linear(input_dim, 10),
    nn.LeakyReLU(0.1),
    nn.Linear(10, 8),
    nn.LeakyReLU(0.1),
    nn.Linear(8, 6),
    nn.LeakyReLU(0.1),
    nn.Linear(6, output_dim)
)

loss_function = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [33]:
indices = torch.randperm(x.size(0)) # 5000개의 인덱스 번호를 랜덤하게 섞는다.
print(indices)
x_batch_list = torch.index_select(x, 0, indices) # shuffle 된 데이터셋으로, 데이터양이 상당하므로, 미니배치 변수로 선언
y_batch_list = torch.index_select(y, 0, indices) # shuffle 된 데이터셋으로, 데이터양이 상당하므로, 미니배치 변수로 선언

print(x_batch_list.shape, y_batch_list.shape) # 데이터를 섞기만 해서 shape는 똑같다.

tensor([4620,  519, 2458,  ..., 4110, 3361, 4613])
torch.Size([5000, 10]) torch.Size([5000, 1])


In [ ]:
x_batch_list = x_batch_list.split(minibatch_size, dim=0) # dim=0 으로 행을 기준으로 5000 /256 = 20
y_batch_list = y_batch_list.split(minibatch_size, dim=0) 
print(len(x_batch_list), len(y_batch_list))

20 20


In [ ]:
print(type(x_batch_list)) # split 으로 나누어서 return 은 tuple이 반환됨
print(type(x_batch_list[0]))
print(x_batch_list[0].shape) 

<class 'tuple'>
<class 'torch.Tensor'>
torch.Size([256, 10])


#### row 레벨로 구현해보기

In [ ]:
for index in range(nb_epochs):
    indices = torch.randperm(x.size(0))
    
    x_batch_list = torch.index_select(x, dim=0, index=indices) # 5000개를 무작위로 shuffle 
    y_batch_list = torch.index_select(y, dim=0, index=indices) # 5000개를 무작위로 shuffle 
    x_batch_list = x_batch_list.split(minibatch_size, dim=0) # 행을 기준으로 minibatch_size 만큼 나눈다.
    y_batch_list = y_batch_list.split(minibatch_size, dim=0) # 행을 기준으로 minibatch_size 만큼 나눈다.
    
    for x_minibatch, y_minibatch in zip(x_batch_list, y_batch_list):
        y_minibatch_pred = model(x_minibatch)
        loss = loss_function(y_minibatch_pred, y_minibatch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    

print(loss)

for param in model.parameters():
    print(param)

tensor(7.9936e-15, grad_fn=<MseLossBackward0>)
Parameter containing:
tensor([[-0.0922,  0.1433,  0.1727, -0.0115, -0.1301, -0.0221, -0.2143, -0.0926,
         -0.0563,  0.2415],
        [-0.2077, -0.0885, -0.1740,  0.0632, -0.0773, -0.2268, -0.2123,  0.2850,
          0.0751,  0.0347],
        [-0.1581,  0.2713,  0.0846, -0.0986,  0.0255,  0.1926,  0.2736, -0.3051,
         -0.2049, -0.1968],
        [ 0.0322, -0.1635, -0.1822, -0.1106,  0.2086, -0.0444, -0.1952,  0.1646,
         -0.1709,  0.2675],
        [ 0.1097, -0.2520,  0.2383, -0.0055, -0.0331, -0.0695, -0.2775, -0.1034,
          0.1557, -0.3065],
        [-0.0510,  0.2764,  0.2185,  0.0529, -0.0005,  0.0539, -0.0418, -0.2067,
          0.0677,  0.2719],
        [ 0.2058,  0.1629,  0.1587, -0.0511, -0.0157,  0.0961,  0.2607,  0.0938,
         -0.2570, -0.0088],
        [ 0.1108,  0.2921, -0.1645, -0.2396,  0.2485, -0.1816, -0.2722, -0.2914,
          0.2913, -0.2421],
        [ 0.2525, -0.3134,  0.0926,  0.1103,  0.2756, -0.10